# TESTS

This document is to test some of the functionality of our main project as well as our exported models. We'll start with the last since it's much more intriguing.

## Test the prepared/pickled models.

All we need here are three imports. Pandas to read the new data, pickle to load the binary field and one import from the python file for our custom pre-processing functions. Actually all we need to do is to load them.

In [1]:
import pickle
import pandas as pd
from functions import custom_clean_transformer, custom_stem_transformer, clean_text, stem_series
from sklearn.metrics import f1_score
import unittest

[nltk_data] Downloading package wordnet to /home/vlado/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Let's read the new data. Here we basically skip the first 80k rows -
# the observation we worked with in our main notebook.The other 83k are completely
# new, unfamiliar to our models.

data = pd.read_csv('data/Twitter_Data.csv', skiprows=80000)
data.columns = ['review', 'sentiment']
data = data.drop_duplicates()
data = data.dropna()
data = data.reset_index()

In [3]:
# Let's read the saved binary objects

classifier_f = open("logistic_regression.pickle", "rb")
logistic_regression = pickle.load(classifier_f)
classifier_f.close()

In [4]:
classifier_f = open("linear_svm.pickle", "rb")
linear_svm = pickle.load(classifier_f)
classifier_f.close()

In [5]:
classifier_f = open("ensemble.pickle", "rb")
ensemble = pickle.load(classifier_f)
classifier_f.close()

That's how you read the binary, pickled object we saved. 

Actually this is it. We opened the pickled file, we loaded the two functions that we wrote to process the new data and we are ready to go.  We can now use our model here with our plain new data and start doing the familiar operations. You can use it in Jupyter, Terminal, Pycharm, VSC, anywhere with python code.

Let's start playing around with our new data.

### Accuracy scores:

In [6]:
print(f"Logistic Regression accuracy: {logistic_regression.score(data['review'], data['sentiment']) *100}%")

Logistic Regression accuracy: 82.59009280462818%


In [7]:
print(f"Linear SVM accuracy: {linear_svm.score(data['review'], data['sentiment']) *100}%")

Linear SVM accuracy: 82.58406653007111%


In [8]:
print(f"Ensemble accuracy: {ensemble.score(data['review'], data['sentiment']) *100}%")

Ensemble accuracy: 83.8965891286007%


### F1 scores:

In [9]:
print(f"Logistic Regression F1: {f1_score(logistic_regression.predict(data['review']), data['sentiment'], average='macro') *100}%")

Logistic Regression F1: 81.23414763582463%


In [10]:
print(f"Linear SVM F1: {f1_score(linear_svm.predict(data['review']), data['sentiment'], average='macro') *100}%")

Linear SVM F1: 81.46042108758674%


In [11]:
print(f"Ensemble F1: {f1_score(ensemble.predict(data['review']), data['sentiment'], average='macro') *100}%")

Ensemble F1: 82.64814419561911%


### Custom tweets

Let's try our models with some 'tweets' written by us.

In [12]:
my_tweets = ['i do not care about him',
             'modi is the worst prime we have had i hope he goes straight to prison when he is done', 
             'we will never find such responsible, successful prime like him',
             'i am running out of ideas',
             'He sucks big time he needs to go now terrible ruler',
             'if you expect to find better leader than him - you are lying yourself'
            ]


In [13]:
print(logistic_regression.predict_proba(my_tweets))
logistic_regression.predict(my_tweets)

[[0.16552075 0.81231821 0.02216103]
 [0.97187613 0.00835537 0.0197685 ]
 [0.00487309 0.00340679 0.99172012]
 [0.06341912 0.87423121 0.06234967]
 [0.96287604 0.0349999  0.00212406]
 [0.00416387 0.00436125 0.99147488]]


array([ 0, -1,  1,  0, -1,  1])

We have the predicted classes and the probabily above.

In [14]:
# # Linear SVM doesn't support predict probability but we've got predict as well
linear_svm.predict(my_tweets)

array([ 0, -1,  1,  0, -1,  1])

In [15]:
print(ensemble.predict_proba(my_tweets))
ensemble.predict(my_tweets)

[[0.07654853 0.89387517 0.0295763 ]
 [0.97783484 0.00247466 0.0196905 ]
 [0.00646574 0.00335635 0.99017791]
 [0.03575313 0.92797214 0.03627473]
 [0.90796465 0.08350527 0.00853008]
 [0.00736976 0.00415462 0.98847562]]


array([ 0, -1,  1,  0, -1,  1])

I gave him pretty easy tweets and it guessed correctly all of them.Feel free to change the list and test it yourself. 

It's a good thing that we achieved the moment when we only load our model, give him raw data and tell him to predict and it does all the things behind - from cleaning and stemming, through the bag of words and tf-idf and finally making the decision.As you can see, we are in completely new environment, we don't have anything imported other than the two libraries and we could have easily skip the pandas import and give him raw scrapped data from twitter for example or whatever. We just confirmed it works - with the same dataset but with unfamiliar observations, it achieved score very close to the original ones.

## Unit tests

Here we'll be checking the functionality of the code we wrote ourselves in the main notebook. Mainly the pre-processing functions 'clean_text' and 'stem_series'.

In [16]:
class FunctionsTests(unittest.TestCase):
    
    def test_cleaning_function_with_dirty_numericData_should_return_cleaned_lower_case(self):
        to_be_given = "1234MyName is dumb"
        actual = clean_text(to_be_given)
        expected = 'myname is dumb'
        self.assertEqual(actual, expected)
        
    
    def test_cleaning_function_with_dirty_data_should_return_cleaned_lower_case(self):
        to_be_given = "$#@$!@#$MyName is dumb"
        actual = clean_text(to_be_given)
        expected = 'myname is dumb'
        self.assertEqual(actual, expected)
    
    
    def test_cleaning_function_with_whiteSpaces_should_return_cleaned_lower_case(self):
        to_be_given = "   Myname is dumb   "
        actual = clean_text(to_be_given)
        expected = 'myname is dumb'
        self.assertEqual(actual, expected)
        
    
    def test_cleaning_function_with_symbols_should_return_cleaned_lower_case(self):
        to_be_given = "Hi I'd like to introduce myself. My name's Ivan"
        actual = clean_text(to_be_given)
        expected = 'hi i would like to introduce myself my name ivan'
        self.assertEqual(actual, expected)
        
    
    def test_cleaning_with_cleaned_should_return_same(self):
        to_be_given = 'hi i am ivan'
        actual = clean_text(to_be_given)
        expected = to_be_given
        self.assertEqual(expected, actual)
        
    
    def test_cleaning_function_with_onlyJunk_should_return_empty(self):
        to_be_given = '21393921 %3!#@*913!@#*    $!#@*!@#(#!)'
        actual = clean_text(to_be_given)
        expected = ""
        self.assertEqual(actual, expected)
        
    
    def test_stemming_function_with_list_with_similar_words_should_return_list_with_one_repeating_word(self):
        to_be_given = ['playing', 'plays', "playful"]
        actual = stem_series(to_be_given)
        expected = ["play "] * len(to_be_given)
        self.assertEqual(actual, expected)
        
        
    def test_stemming_function_different_type_words_should_stem_and_return_stemmed(self):
        to_be_given = ['television', 'computer']
        actual = stem_series(to_be_given)
        expected = ['televis ', 'comput ']
        self.assertEqual(actual, expected)
        
        
    def test_stemming_with_already_stemmedList_should_return_the_same_list(self):
        to_be_given = ['televi', "comput"]
        actual = stem_series(to_be_given)
        expected = ['televi ', 'comput ']
        self.assertEqual(actual, expected)
    
    def test_stemming_with_empty_should_return_empty(self):
        to_be_given = []
        actual = stem_series(to_be_given)
        expected = []
        self.assertEqual(actual, expected)
        
        
    
unittest.main(argv=[''], verbosity=2, exit=False)


test_cleaning_function_with_dirty_data_should_return_cleaned_lower_case (__main__.FunctionsTests) ... ok
test_cleaning_function_with_dirty_numericData_should_return_cleaned_lower_case (__main__.FunctionsTests) ... ok
test_cleaning_function_with_onlyJunk_should_return_empty (__main__.FunctionsTests) ... ok
test_cleaning_function_with_symbols_should_return_cleaned_lower_case (__main__.FunctionsTests) ... ok
test_cleaning_function_with_whiteSpaces_should_return_cleaned_lower_case (__main__.FunctionsTests) ... ok
test_cleaning_with_cleaned_should_return_same (__main__.FunctionsTests) ... ok
test_stemming_function_different_type_words_should_stem_and_return_stemmed (__main__.FunctionsTests) ... ok
test_stemming_function_with_list_with_similar_words_should_return_list_with_one_repeating_word (__main__.FunctionsTests) ... ok
test_stemming_with_already_stemmedList_should_return_the_same_list (__main__.FunctionsTests) ... ok
test_stemming_with_empty_should_return_empty (__main__.FunctionsTests)